If on google collab, if not skip the following cell

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/TFG_Matematicas'

Mounted at /content/gdrive
/content/gdrive/MyDrive/TFG_Matematicas


## Preparation to test models

In [24]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dense, Dropout, Input, SimpleRNN, LSTM, Input, BatchNormalization, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import plot_model
from utils.utils import *


In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
print("All devices: ", tf.config.list_logical_devices('TPU'))
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [3]:

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
train_idxs = read_indexes('./utils/train.txt')
validation_idxs =  read_indexes('./utils/validation.txt')
test_idxs = read_indexes('./utils/test.txt')

In [5]:
data_indexes = [(i+1,j+1) for i in range(10) for j in range(2000)]
parameters = load_parameter_list()
df_parameters = pd.DataFrame(parameters, columns=['cx', 'cy', 'a', 'b', 'theta', 'e1', 'e2'])
df_parameters['data_index'] = data_indexes

y_train = df_parameters[df_parameters['data_index'].isin(train_idxs)].drop('data_index', axis = 1)
y_validation = df_parameters[df_parameters['data_index'].isin(validation_idxs)].drop('data_index', axis = 1)
y_test = df_parameters[df_parameters['data_index'].isin(test_idxs)].drop('data_index', axis = 1)

In [6]:
X_train = np.array(load_data_multithreaded(train_idxs,100, small = True))
print('train loaded')
X_validation = np.array(load_data_multithreaded(validation_idxs,100, small = True))
print('validation loaded')
#X_test = np.array(load_data_multithreaded(test_idxs,100, small = True))
#print('test loaded')

train loaded
validation loaded


In [7]:
mean = np.mean(X_train.flatten())
std = np.std(X_train.flatten())

In [8]:
mean, std

(0.00289415908159017, 0.015852358244609798)

# Preparing metrics

In [9]:
def e1_mse(y_true, y_pred):
    output = y_pred[:, -2]
    output_mae = tf.keras.metrics.mean_squared_error(y_true[:, -2], output)
    return output_mae
def e2_mse(y_true, y_pred):
    output = y_pred[:, -1]
    output_mae = tf.keras.metrics.mean_squared_error(y_true[:, -1], output)
    return output_mae

def e1_mae(y_true, y_pred):
    output = y_pred[:, -2]
    output_mae = tf.keras.metrics.mean_absolute_error(y_true[:, -2], output)
    return output_mae
def e2_mae(y_true, y_pred):
    output = y_pred[:, -1]
    output_mae = tf.keras.metrics.mean_absolute_error(y_true[:, -1], output)
    return output_mae

def e1_rmse(y_true, y_pred):
    output = y_pred[:, -2]
    output_rmse = rmse(y_true[:, -2], output)
    return output_rmse
def e2_rmse(y_true, y_pred):
    output = y_pred[:, -1]
    output_rmse = rmse(y_true[:, -1], output)
    return output_rmse

In [10]:
METRICS=[tf.keras.metrics.RootMeanSquaredError(name='rmse'),
         tf.keras.metrics.MeanAbsoluteError(name='mae'),
         e1_mse,e2_mse,e1_mae,e2_mae]


#### Libraries for hyperparameter tunning

In [11]:
pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [12]:
import keras_tuner as kt

# Feedforward model

In [26]:
def model_builder(hp):

    # hyperparameter space
    layer_combinations = {
        3: [1024, 256, 64],
        4: [1024, 256, 128, 64],
        5: [1024, 512, 256, 128, 64],
    }
    n = hp.Choice('layers', values=[3,4,5])
    dropout_boolean = hp.Choice('dropout', values=[True, False])
    lr = hp.Choice('learning_rate', values=[0.001,0.0001])
    activation = hp.Choice('activation', values=["relu","leaky_relu"])

    # model construction
    feedforward = keras.Sequential()
    feedforward.add(Input(shape=(440, 17)))
    feedforward.add(Flatten())
    for i in layer_combinations[n]:
      feedforward.add(Dense(i, activation=activation))
      if dropout_boolean:
        feedforward.add(Dropout(0.2))
    feedforward.add(Dense(7))

    # model compilation
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    feedforward.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return feedforward

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    directory='hyperparameter_tunning',
    project_name='feedforward'
)

tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

Trial 28 Complete [00h 00m 45s]
val_loss: 0.6830689907073975

Best val_loss So Far: 0.5637280941009521
Total elapsed time: 00h 07m 35s


In [27]:
tuner.results_summary()

Results summary
Results in hyperparameter_tunning/feedforward
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0016 summary
Hyperparameters:
layers: 3
dropout: 0
learning_rate: 0.001
activation: leaky_relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.5637280941009521

Trial 0017 summary
Hyperparameters:
layers: 3
dropout: 0
learning_rate: 0.001
activation: relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.568383514881134

Trial 0024 summary
Hyperparameters:
layers: 4
dropout: 0
learning_rate: 0.001
activation: leaky_relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 0.5703068971633911

Trial 0025 summary
Hyperparameters:
layers: 4
dropout: 0
learning_rate: 0.001
activation: relu
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0020
Score: 0.5707733035087585


# Convolutional model

In [ ]:
def model_builder(hp):

    # hyperparameter space
    lr = hp.Choice('learning_rate', values=[0.001,0.0001])
    activation = hp.Choice('activation', values=["relu","leaky_relu"])
    conv_n = hp.Choice('conv_layers', values=[2,3])
    kernel_s =  hp.Choice('kernel_size', values=[3,5])
    kernel_sizes = {
        3: (3,3),
        5: (5,5)
    }
    layer_combinations = {
          3: [1024, 256, 64],
          4: [1024, 256, 128, 64],
          5: [1024, 512, 256, 128, 64],
      }
    n = hp.Choice('layers', values=[3,4,5])
    pooling_layers = hp.Choice('layers', values=['max', 'avg'])
    transition_layer = hp.Choice('layers', values=['g_max', 'g_avg', 'flatten'])
    activation = hp.Choice('learning_rate', values=["relu","leaky_relu"])
    dropout_boolean = hp.Choice('dropout', values=[True, False])

    # model construction
    cnn = keras.Sequential()
    cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(440, 17, 1), padding='same'))
    cnn.add(BatchNormalization()),
    if pooling_layers == 'max':
        cnn.add(MaxPooling2D((1, 2))),
    else:
      cnn.add(AveragePooling2D((1, 2))),

    for i in range(conv_n-1):
      cnn.add(Conv2D(32, kernel_sizes[kernel_s], activation='relu', input_shape=(440, 17, 1), padding='same'))
      cnn.add(BatchNormalization()),
      if pooling_layers == 'max':
        cnn.add(MaxPooling2D((1, 2))),
      else:
        cnn.add(AveragePooling2D((1, 2))),

    if transition_layer == 'g_max':
      cnn.add(GlobalMaxPooling2D()),
    elif transition_layer == 'g_avg':
      cnn.add(GlobalAveragePooling2D()),
    else:
      cnn.add(Flatten()),

    for i in layer_combinations[n]:
      cnn.add(Dense(i, activation=activation))
      if dropout_boolean:
        cnn.add(Dropout(0.2))
    cnn.add(Dense(7))

    #model compilation
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    cnn.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return cnn

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    directory='hyperparameter_tunning',
    project_name='cnn'
)

tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

In [ ]:
tuner.results_summary()

# Recurrent Neural Network with simple RNN layer

In [35]:
def model_builder(hp):

    # hyperparameter space
    rnn_layer_combinations = {
          1: [20],
          2: [30],
          3: [20, 30],
      }
    rnn_n = hp.Choice('rnn_layers', values=[1,2,3])
    layer_combinations = {
        3: [1024, 256, 64],
        4: [1024, 256, 128, 64],
        5: [1024, 512, 256, 128, 64],
    }

    n = hp.Choice('layers', values=[3,4,5])
    dropout_boolean = hp.Choice('dropout', values=[True, False])
    sequences_boolean = hp.Choice('sequences', values=[True, False])
    lr = hp.Choice('learning_rate', values=[0.001,0.0001])
    activation = hp.Choice('activation', values=["relu","leaky_relu"])

    # model construction
    if len(rnn_layer_combinations[rnn_n]) == 1:
      rnn = SimpleRNN(rnn_layer_combinations[rnn_n][0], input_shape=(440, 17), return_sequences=sequences_boolean)
      rnn.add(Flatten())
    else:
      rnn = SimpleRNN(rnn_layer_combinations[rnn_n][0], input_shape=(440, 17), return_sequences=True)
      rnn.add(SimpleRNN(rnn_layer_combinations[rnn_n][1], return_sequences=sequences_boolean))

    rnn.add(Flatten())
    for i in layer_combinations[n]:
      rnn.add(Dense(i, activation=activation))
      if dropout_boolean:
        rnn.add(Dropout(0.2))
    rnn.add(Dense(7))

    # model compilation
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    rnn.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return rnn

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    directory='hyperparameter_tunning',
    project_name='rnn'
)



tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

SyntaxError: expected ':' (<ipython-input-35-7de0b660ef14>, line 36)

In [ ]:
tuner.results_summary()

#  Recurrent Neural Network Model with LSTM

In [ ]:
def model_builder(hp):

    # hyperparameter space
    lstm_layer_combinations = {
          1: [20],
          2: [30],
          3: [20, 30],
      }
    lstm_n = hp.Choice('lstm_layers', values=[1,2,3])
    layer_combinations = {
        3: [1024, 256, 64],
        4: [1024, 256, 128, 64],
        5: [1024, 512, 256, 128, 64],
    }

    n = hp.Choice('layers', values=[3,4,5])
    dropout_boolean = hp.Choice('dropout', values=[True, False])
    sequences_boolean = hp.Choice('sequences', values=[True, False])
    lr = hp.Choice('learning_rate', values=[0.001,0.0001])
    activation = hp.Choice('activation', values=["relu","leaky_relu"])

    # model construction
    if len(lstm_layer_combinations[lstm_n]) == 1:
      rnn_lstm = SimpleRNN(lstm_layer_combinations[lstm_n][0], input_shape=(440, 17), return_sequences=sequences_boolean)
      rnn_lstm.add(Flatten())
    else:
      rnn_lstm = SimpleRNN(lstm_layer_combinations[lstm_n][0], input_shape=(440, 17), return_sequences=True)
      rnn_lstm.add(SimpleRNN(lstm_layer_combinations[lstm_n][1], return_sequences=sequences_boolean))

    rnn_lstm.add(Flatten())
    for i in layer_combinations[n]:
      rnn_lstm.add(Dense(i, activation=activation))
      if dropout_boolean:
        rnn_lstm.add(Dropout(0.2))
    rnn_lstm.add(Dense(7))

    # model compilation
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    rnn_lstm.compile(optimizer=optimizer, loss='mse', metrics=[METRICS])

    return rnn

tuner = kt.Hyperband(
    model_builder,
    objective='val_loss',
    max_epochs=25,
    directory='hyperparameter_tunning',
    project_name='rnn_lstm'
)



tuner.search(X_train, y_train, validation_data=(X_validation, y_validation))

In [ ]:
tuner.results_summary()